
# Notebook de Ingestão de Dados - Camada Bronze

Este notebook tem como objetivo realizar a ingestão de dados para a camada bronze, responsável por armazenar os dados brutos provenientes das fontes originais. Os dados adicionados neste processo são provenientes do portal de transparência do governo federal.

Serão armazenados dois conjuntos de dados:
- As emendas parlamentares, referentes aos anos de 2023 até 2025, que foram adicionadas diretamente pelo carregamento de três arquivos CSV na camada staging.
- Os parlamentares em exercício de 2023 a 2026, obtidos via requisições e ingeridos neste notebook.

# Configuração do Ambiente

## Carregando bibliotecas

In [0]:
import requests
from PIL import Image
import unicodedata
from IPython.display import display

# Criando tabelas com os dados

## Parlamentares

In [0]:
# Url para requisição de deputados em exercício
url = "https://dadosabertos.camara.leg.br/api/v2/deputados"

# Pegando todos os deputados em exercício (mandato atual)
params = {
    "ordem": "ASC",
    "ordenarPor": "nome",
    "pagina": 1,
    "itens": 100
}

todos = []
while True:
    resp = requests.get(url, params=params)
    resp.raise_for_status()

    dados = resp.json()["dados"]
    todos.extend(dados)

    # Verifica se acabou a paginação
    if "links" in resp.json():
        next_link = [lnk for lnk in resp.json()["links"] if lnk["rel"] == "next"]
        if next_link:
            params["pagina"] += 1
        else:
            break
    else:
        break

# Converter JSON para Spark DataFrame
df_parlamentares = spark.createDataFrame(todos)

# Gravar no bronze (Delta)
df_parlamentares.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("mvp.bronze.parlamentares")


In [0]:
df_parlamentares.count()

In [0]:
df_parlamentares.limit(5).display()

In [0]:
%sql
SELECT *
FROM mvp.bronze.parlamentares
WHERE email LIKE "%glauber%"

O deputado que não está contido na base foi o deputado que foi suspenso pela câmara dos deputados

https://g1.globo.com/politica/noticia/2025/12/10/camara-suspende-por-seis-meses-deputado-glauber-braga-deputado-era-alvo-de-processo-de-cassacao.ghtml


> Tabela criada com sucesso contendo 512 linhas, correspondente ao número de deputados federais no Brasil.  
> Para esses deputados, foram ingeridas as seguintes colunas:
> - email
> - id
> - idLegislatura (57 é o mandato atual do Brasil)
> - nome
> - siglaPartido
> - siglaUF
> - uri
> - uriPartido
> - uriFoto

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.bronze.parlamentares IS 
'Tabela de parlamentares em exercício na legislatura 57 (2023-2026), contendo informações como nome, partido, UF, email, foto e outros dados relevantes.'
""")

## Emendas


As emendas parlamentares foram adicionadas a camada staging do catalógo do mvp e será carregada no schema bronze por meio do spark.

In [0]:
df_emendas = (
    spark.read
        .option("header", True)
        .option("sep", ";")
        .csv("dbfs:/Volumes/mvp/staging/emendas/emendas_*.csv")
)

In [0]:
df_emendas.count()

In [0]:
df.limit(5).display()

In [0]:

# Tratamento de texto das colunas para serem salvas na camada bronze
def remover_acentos(col):
    return ''.join(
        c for c in unicodedata.normalize('NFKD', col)
        if not unicodedata.combining(c)
    )

df = df.toDF(*[
    remover_acentos(col)
    .replace(' ', '_')
    .replace('/', '_')
    .replace('(', '_')
    .replace(')', '_')
    .replace('?', '_')
    for col in df.columns
])
df.write.format("delta").mode("overwrite").saveAsTable("mvp.bronze.emendas")

In [0]:
%sql
select *
from mvp.bronze.emendas
limit 5

> A carga das emendas parlamentares foi realizada corretamente, contendo 18739 linhas e as seguintes colunas:
> - Ano: Foram carregados os anos 2023, 2024 e 2025
> - Tipo de Emenda
> - Autor da emenda
> - Número da emenda
> - Possui Apoiador/Solicitante?
> - Localidade do gasto (Regionalização)
> - Função
> - Subfunção
> - Programa Orçamentário
> - Ação Orçamentária
> - Plano Orçamentário
> - Código da emenda
> - Valor empenhado
> - Valor liquidado
> - Valor pago
> - Valor Restos a Pagar Inscritos
> - Valor Restos a Pagar Cancelados
> - Valor Restos a Pagar Pagos
> - _c18

In [0]:
spark.sql("""
COMMENT ON TABLE mvp.bronze.emendas IS 
'Tabela de emendas parlamentares do período de 2023 até 2025, contendo informações sobre os gastos realizados, o parlamentar autor da emenda, a função associada, além de outros detalhes relevantes das emendas.'
""")